<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Connect-to-DB-(Extract)" data-toc-modified-id="Connect-to-DB-(Extract)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Connect to DB (Extract)</a></span></li><li><span><a href="#Create-Dimensions-(Transform)" data-toc-modified-id="Create-Dimensions-(Transform)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create Dimensions (Transform)</a></span><ul class="toc-item"><li><span><a href="#Location" data-toc-modified-id="Location-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Location</a></span></li><li><span><a href="#Parameter" data-toc-modified-id="Parameter-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Parameter</a></span></li><li><span><a href="#DateTime" data-toc-modified-id="DateTime-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>DateTime</a></span></li></ul></li><li><span><a href="#Upload-to-Database-(Load)" data-toc-modified-id="Upload-to-Database-(Load)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Upload to Database (Load)</a></span></li></ul></div>

In [236]:
import pandas as pd
import psycopg2 as pg
from datetime import date, datetime
import time
from sqlalchemy import create_engine, text, inspect
import requests
import json
import datetime
import logging
import pytz
import warnings

### Connect to DB (Extract)

In [42]:
# DB credentials
ENDPOINT = 'lakehousedwh.cejfhcomb8d8.us-east-1.rds.amazonaws.com'
DB_NAME = 'lakehousedwh'
USERNAME = 'trunks'
PASSWORD = 'Corn5!!!'


# connection string for sql alchemy engine
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DB_NAME}'
    
# create sql alchemy engine
db = create_engine(conn_string)
dbconnect = db.connect()

In [47]:
df_weather_traffic_stage = pd.read_sql_table('weather_traffic_stage', dbconnect)
df_oil_stage = pd.read_sql_table('oil_stage', dbconnect)

In [237]:
df_weather_traffic_stage

,DatetimeUTC,Location,Parameter,Value,Unit
0,2022-04-24 21:00:00+00:00,Stampfenbachstrasse,MIV,191.00,Count
1,2022-04-24 21:00:00+00:00,Schimmelstrasse,MIV,998.00,Count
2,2022-04-24 21:00:00+00:00,Rosengartenstrasse,MIV,2149.00,Count
3,2022-04-24 20:00:00+00:00,Stampfenbachstrasse,MIV,258.00,Count
4,2022-04-24 20:00:00+00:00,Schimmelstrasse,MIV,1198.00,Count
...,...,...,...,...,...
720540,2013-01-01 00:00:00+00:00,Schimmelstrasse,MIV,1189.00,Count
720541,2013-01-01 00:00:00+00:00,Rosengartenstrasse,MIV,2090.00,Count
720542,2013-01-01 00:00:00+00:00,Schimmelstrasse,RainDur,0.00,min
720543,2013-01-01 00:00:00+00:00,Schimmelstrasse,T,2.04,°C


### Create Dimensions (Transform)

#### Location

In [240]:
dim_location = df_weather_traffic_stage[['Location']]
dim_location = dim_location.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False).reset_index(drop=True) #only unique vlaues
dim_location.insert(loc=0, column='id', value=dim_location.index + 1) #ad id column

In [241]:
dim_location

,id,Location
0,1,Stampfenbachstrasse
1,2,Schimmelstrasse
2,3,Rosengartenstrasse


#### Parameter

In [242]:
dim_parameter_1 = df_weather_traffic_stage[['Parameter','Unit']]
dim_parameter_1 = dim_parameter_1.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False).reset_index(drop=True) #only unique vlaues

In [243]:
dim_parameter_2 = df_oil_stage[['Parameter','Unit']]
dim_parameter_2 = dim_parameter_2.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False).reset_index(drop=True) #only unique vlaues

In [244]:
dim_parameter

,id,Parameter,Unit
0,1,MIV,Count
1,2,RainDur,min
2,3,T,°C
3,4,Price,$


#### DateTime

In [230]:
dim_datetime = df_weather_traffic_stage[["DatetimeUTC"]].drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False).reset_index(drop=True) #only unique vlau

In [231]:
#dim_datetime['DatetimeSWISS'] = dim_datetime['DatetimeUTC'].dt.tz_convert('Europe/Zurich')
dim_datetime['Date'] = dim_datetime['DatetimeUTC'].dt.date
dim_datetime['Time'] = dim_datetime['DatetimeUTC'].dt.time
dim_datetime['DayOfWeek'] = dim_datetime['DatetimeUTC'].dt.dayofweek
dim_datetime['DayName'] = dim_datetime['DatetimeUTC'].dt.day_name()
dim_datetime['Month'] = dim_datetime['DatetimeUTC'].dt.month
dim_datetime['Week'] = dim_datetime['DatetimeUTC'].dt.isocalendar().week
dim_datetime['Quarter'] = dim_datetime['DatetimeUTC'].dt.quarter
dim_datetime['Date']= pd.to_datetime(dim_datetime['Date'])

In [232]:
dim_datetime

,DatetimeUTC,Date,Time,DayOfWeek,DayName,Month,Week,Quarter
0,2022-04-24 21:00:00+00:00,2022-04-24,21:00:00,6,Sunday,4,16,2
1,2022-04-24 20:00:00+00:00,2022-04-24,20:00:00,6,Sunday,4,16,2
2,2022-04-24 19:00:00+00:00,2022-04-24,19:00:00,6,Sunday,4,16,2
3,2022-04-24 18:00:00+00:00,2022-04-24,18:00:00,6,Sunday,4,16,2
4,2022-04-24 17:00:00+00:00,2022-04-24,17:00:00,6,Sunday,4,16,2
...,...,...,...,...,...,...,...,...
81615,2013-01-01 04:00:00+00:00,2013-01-01,04:00:00,1,Tuesday,1,1,1
81616,2013-01-01 03:00:00+00:00,2013-01-01,03:00:00,1,Tuesday,1,1,1
81617,2013-01-01 02:00:00+00:00,2013-01-01,02:00:00,1,Tuesday,1,1,1
81618,2013-01-01 01:00:00+00:00,2013-01-01,01:00:00,1,Tuesday,1,1,1


### Upload to Database (Load)

In [245]:
dim_location.to_sql('dim_location', engine, if_exists='replace',index=False, method='multi')
dim_parameter.to_sql('dim_parameter', engine, if_exists='replace',index=False, method='multi')
dim_datetime.to_sql('dim_datetime', engine, if_exists='replace',index=False, method='multi',chunksize=5000)

In [235]:
df_weather_traffic_stage.to_sql('fact_messures', engine, if_exists='replace',index=False, method='multi',chunksize=5000)
df_oil_stage.to_sql('fact_prices', engine, if_exists='replace',index=False, method='multi',chunksize=5000)
